# Подключаем все необходимые библиотеки, и подготавливаем данные

In [ ]:
from __future__ import print_function
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from imgaug.augmentables.segmaps import SegmentationMapsOnImage
import numpy as np
import pandas as pd
import keras
from sklearn.model_selection import train_test_split
from keras.preprocessing import image
from keras.models import Model,load_model
import segmentation_models as sm
from math import ceil
import tensorflow as tf
from math import ceil
import os
import sys
from sklearn.linear_model import LinearRegression
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

In [ ]:
batch_size = 10
filesDir = "E:/ittensive/ML Neural networks/Part 4/train_images_small"
image_x = 384 # 525
image_y = 256 # 350
image_ch = 3 # 3
mask_x = 384
mask_y = 256
def mask_rate (a, x, y):
    b = a//1400 + 0.0
    return np.round(x*(b*x//2100) + y*(a%1400)//1400).astype("uint32")

def calc_mask (px, x=image_x, y=image_y):
    p = np.array([int(n) for n in px.split(' ')]).reshape(-1,2)
    mask = np.zeros(x*y, dtype='uint8')
    for i, l in p:
        mask[mask_rate(i, x, y) - 1:mask_rate(l+i, x, y)] = 1
    return mask.reshape(y,x).transpose()

def calc_dice (x):
    dice = 0
    px = x["EncodedPixels"] 
    if px != px and x["target"] == 0:
        dice = 1
    elif px == px and x["target"] == 1:
        mask = calc_mask(px).flatten()
        target = np.array(x["TargetPixels"].split(" ")).astype("int8")
        dice += 2*np.sum(target[mask==1])/(np.sum(target)+np.sum(mask))
    return dice

def load_y (df):
    y = [[0]]*len(df)
    for i, ep in enumerate(df["EncodedPixels"]):
        if ep == ep:
            y[i] = calc_mask(ep, mask_x, mask_y).transpose().flatten()
        else:
            y[i] = np.zeros(mask_x*mask_y, dtype="i1")
    return np.array(y).reshape(len(df), mask_y, mask_x, 1)

def load_x (df):
    x = [[]]*len(df)
    for j, file in enumerate(df["Image"]):
        img = image.load_img(os.path.join(filesDir, file),
                     target_size=(image_y, image_x))
        img = image.img_to_array(img)
        x[j] = np.expand_dims(img, axis=0)
    return np.array(x).reshape(len(df), image_y, image_x, image_ch)

def load_data (df, batch_size):
    while True:
        batch_start = 0
        batch_end = batch_size
        while batch_start < len(df):
            limit = min(batch_end, len(df))
            yield (load_x(df[batch_start:limit]),
                   load_y(df[batch_start:limit]))
            batch_start += batch_size   
            batch_end += batch_size

def draw_prediction (prediction):
    fig = plt.figure(figsize=(16, 8))
    ax = fig.add_subplot(1,1,1)
    ax.hist(prediction[0])
    ax.set_title("Fish")
    plt.show()
    
def multiply(prediction, argument):
    result_list = []
    for multiplied, i  in enumerate(prediction):
        result =  prediction[multiplied] * argument[multiplied]
        result_list.append(result)
    return result_list

In [ ]:
data = pd.read_csv('E:/ittensive/ML Neural networks/Part 4/train.csv.gz')

In [ ]:
data["Image"] = data["Image_Label"].str.split("_").str[0]
data["Label"] = data["Image_Label"].str.split("_").str[1]
data.drop(labels=["Image_Label"], axis=1, inplace=True)
data_fish = data[data["Label"] == "Fish"]
print (data_fish.head())

In [ ]:
train, test = train_test_split(data_fish, test_size=0.2)
train = pd.DataFrame(train)
test = pd.DataFrame(test)
del data
print (train.head())

## Загружаем модели нейросетей, и обучаем их 

# U-net (train)

In [ ]:
model_unet = load_model("E:/ittensive/ML Neural networks/Part 4/U-net/unet.fish.h5", compile=False)

In [ ]:
prediction_unet = model_unet.predict_generator(load_data(train, 1), steps=len(train), verbose=1)

In [ ]:
prediction_unet = prediction_unet.astype('int8')

In [ ]:
np.save('E:/ittensive/ML Neural networks/Part 4/U-net/prediction_unet.npy', prediction_unet)

# FPN (train)

In [ ]:
model_fpn = load_model("E:/ittensive/ML Neural networks/Part 4/FPN/fpn.fish.h5", compile=False)

In [ ]:
prediction_fpn = model_fpn.predict_generator(load_data(train, 1), steps=len(train), verbose=1)

In [ ]:
prediction_fpn = prediction_fpn.astype('int8')

In [ ]:
np.save('E:/ittensive/ML Neural networks/Part 4/FPN/prediction_fpn.npy', prediction_fpn)

# PSPnet (train)

In [ ]:
model_fpn = load_model("E:/ittensive/ML Neural networks/Part 4/FPN/fpn.fish.h5", compile=False)

In [ ]:
prediction_pspnet = model_pspnet.predict_generator(load_data(train, 1), steps=len(train), verbose=1)

In [ ]:
prediction_pspnet = prediction_pspnet.astype('int8')

In [ ]:
np.save('E:/ittensive/ML Neural networks/Part 4/PSPnet/prediction_pspnet.npy', prediction_pspnet)

# Распаковываем маску изображений

In [ ]:
train["EncodedPixels"] = train["EncodedPixels"].fillna(0)

list_intrepolite = [0]
list_intrepolite = list_intrepolite * 384
list_intrepolite = np.array(list_intrepolite).reshape(384, 1).astype('int8')


mask_intepolite = []
for pixel in range(0, 256):
    mask_intepolite.append(list_intrepolite)

    
new_frame = []    
for row in train["EncodedPixels"]:
    if row != 0:
        new_frame.append(np.array(calc_mask(row)).reshape(384, 256, 1).astype('int8'))
    else:
        new_frame.append(np.array(mask_intepolite).astype('int8'))
        
np.save('E:/ittensive/ML Neural networks/Part 4/new_frame.npy', new_frame)

# Строим модель CatBoost

In [ ]:
y = np.array(new_frame).flatten().reshape(436076544,1)

In [ ]:
x = np.array([prediction_unet.reshape(436076544,1),
                              prediction_fpn.reshape(436076544,1), 
                              prediction_pspnet.reshape(436076544,1)])

x = np.transpose(x)
x = x[0]

In [ ]:
del prediction_unet
del prediction_fpn
del prediction_pspnet

train_dataset = Pool(data=x, label=y)
model = CatBoostClassifier(iterations=10, learning_rate=0.57,
                random_seed=17, depth=6, loss_function="MultiClass",
                bootstrap_type="MVS", custom_metric="WKappa")

In [ ]:
cb_params = {
    "depth": range(5,8),
    'learning_rate': np.arange(0.56,0.59,0.01),
    'l2_leaf_reg': range(1,5),}

cb_grid = model.grid_search(cb_params, cv=5, X=x, y=y, verbose=True)
print (cb_grid["params"])

In [ ]:
model = CatBoostClassifier(iterations=100,
        learning_rate=cb_grid["params"]["learning_rate"],
        depth=cb_grid["params"]["depth"],
        l2_leaf_reg=cb_grid["params"]["l2_leaf_reg"],
        random_seed=17, loss_function="MultiClass",
        bootstrap_type="MVS", custom_metric="WKappa")

In [ ]:
model.fit(train_dataset)

In [ ]:
del x 
del y

# Создаём тестовые данные

# U-net test

In [ ]:
model_unet = load_model("E:/ittensive/ML Neural networks/Part 4/U-net/unet.fish.h5", compile=False)

In [ ]:
prediction_unet = model_unet.predict_generator(load_data(test, 1), steps=len(test), verbose=1)

In [ ]:
prediction_unet = prediction_unet.astype('int8')

In [ ]:
np.save('E:/ittensive/ML Neural networks/Part 4/FPN/fpn_test.npy', prediction_fpn)

# FPN test

In [ ]:
model_fpn = load_model("E:/ittensive/ML Neural networks/Part 4/FPN/fpn.fish.h5", compile=False)

In [ ]:
prediction_fpn = model_fpn.predict_generator(load_data(test, 1), steps=len(test), verbose=1)

In [ ]:
prediction_fpn = prediction_fpn.astype('int8')

In [ ]:
np.save('E:/ittensive/ML Neural networks/Part 4/FPN/fpn_test.npy', prediction_fpn)

# PSPnet test

In [ ]:
model_pspnet = load_model("E:/ittensive/ML Neural networks/Part 4/PSPnet/pspnet.fish.h5", compile=False)

In [ ]:
prediction_pspnet = model_pspnet.predict_generator(load_data(test, 1), steps=len(test), verbose=1)

In [ ]:
prediction_pspnet = prediction_pspnet.astype('int8')

In [ ]:
np.save('E:/ittensive/ML Neural networks/Part 4/PSPnet/pspnet_test.npy', prediction_pspnet)

In [ ]:
del prediction_unet
del prediction_fpn
del prediction_pspnet

# Добавления тестовых данных в CatBoost

In [ ]:
test_unet = np.load('E:/ittensive/ML Neural networks/Part 4/U-net/unet_test.npy') 
test_unet = test_unet.astype('int8').flatten()

In [ ]:
test_fpn = np.load('E:/ittensive/ML Neural networks/Part 4/FPN/fpn_test.npy') 
test_fpn = test_fpn.astype('int8').flatten()

In [ ]:
test_pspnet = np.load('E:/ittensive/ML Neural networks/Part 4/PSPnet/pspnet_test.npy') 
test_pspnet = test_pspnet.astype('int8').flatten()

In [ ]:
test_list_1 = np.array([test_unet.reshape(109117440,1), test_fpn.reshape(109117440,1), 
                        test_pspnet.reshape(109117440,1)])
test_list_1 = np.transpose(test_list_1)

In [ ]:
del test_unet
del test_fpn
del test_pspnet

# Получаем итоговый результат

In [ ]:
target = model.predict(Pool(data=test_list_1[0]))

In [ ]:
target = target.astype('int8')

In [ ]:
np.save('E:/ittensive/ML Neural networks/Part 4/catboost/catboost.npy', target)